# Answers and Analysis/Findings for Thursday Board Meeting

In [1]:
# Importing necessary tools and files
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics


import joint_acquire
import joint_prepare
import env


In [2]:
# Utilizing acquire.py to assign dataset to `df`
df = joint_acquire.acquire_logs(user=env.user, password=env.password, host=env.host)
df

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0


In [3]:
# Utilizing prepare.py to clean and prep dataset;
# Separated data with lack of context into `df_admin`
df, df_admin = joint_prepare.prepare_logs(df)

In [4]:
# Expanding view of dataframes and information in this notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

-----------------------------------------------------------------------------------------------------------
## 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [5]:
# Dataframe Variable Overview
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847329 entries, 0 to 900222
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date           847329 non-null  object 
 1   time           847329 non-null  object 
 2   path           847329 non-null  object 
 3   user_id        847329 non-null  int64  
 4   cohort_id      847329 non-null  float64
 5   ip             847329 non-null  object 
 6   name           847329 non-null  object 
 7   start_date     847329 non-null  object 
 8   end_date       847329 non-null  object 
 9   created_at     847329 non-null  object 
 10  program_id     847329 non-null  float64
 11  primary_topic  847329 non-null  object 
 12  subtopic       847329 non-null  object 
 13  tertiary       847329 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 129.2+ MB


In [6]:
# Dataframe Overview
df.head()

,date,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,program_id,primary_topic,subtopic,tertiary
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,1.0,,,None
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,1.0,java-ii,None,None
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,1.0,java-ii,object-oriented-programming,None
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,1.0,slides,object_oriented_programming,None
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2.0,javascript-i,conditionals,None


In [7]:
# Splitting dataframe by program into 4 separate dfs
full_stack_php = df[df.program_id == 1]
full_stack_java = df[df.program_id == 2]
data_science = df[df.program_id == 3]
front_end = df[df.program_id == 4]

### Full-Stack PHP Program

In [8]:
# Full-Stack PHP: Count of Modules Accessed by Cohort
php_primary_count = (full_stack_php.groupby('name').primary_topic.value_counts())

In [9]:
# Full-Stack PHP: Percentage of Modules Accessed by Cohort
php_primary_count_pct = (full_stack_php.groupby('name').primary_topic.value_counts(normalize=True))

In [10]:
# Dataframe of Count of Modules Accessed by Cohort
pd.DataFrame(php_primary_count)

primary_topic
name       primary_topic                                       
Arches     javascript-i                           1333         
           html-css                               1031         
           spring                                 914          
           mysql                                  775          
           java-iii                               696          
           javascript-ii                          672          
           jquery                                 650          
                                                  626          
           java-ii                                500          
           java-i                                 422          
           appendix                               393          
           content                                277          
           slides                                 219          
           examples                               121          
           toc                                    66           
           mkdocs                                 56           
           prework                                55           
           search                                 45           
           assets                                 9            
           cohorts                                9            
           5.0_Intro_Regression                   2            
           7.0_Intro_Clustering                   2            
           register.html                          2            
           registration                           2            
           students                               2            
           uploads                                2            
           0_Classification_Algorithms            1            
           2.0_Intro_Stats                        1            
           8.0_Intro_Module                       1            
           About_NLP                              1            
           Module_6_Classification                1            
           index.html                             1            
           introduction-to-matplotlib             1            
           login                                  1            
           pre-work                               1            
Badlands   content                                57           
                                                  17           
           prework                                6            
           javascript-ii                          5            
           html-css                               4            
           toc                                    2            
           java-i                                 1            
           javascript-i                           1            
Denali     prework                                2            
                                                  1            
           mkdocs                                 1            
Everglades                                        1            
Franklin   javascript-ii                          13           
           java-iii                               11           
           javascript-i                           8            
           java-ii                                5            
           mysql                                  5            
           spring                                 5            
                                                  4            
           java-i                                 4            
           jquery                                 3            
           11._DistributedML                      1            
           12.01_SocialNetworkAnalysis            1            
           2.00.02_Navigating_Excel               1            
           2.00.05_Charts_PivotTables_Sparklines  1            
           2.02.00_Inferential_Stats              1            
           3.9-tem

In [11]:
# Dataframe of Percentage of Modules Accessed by Cohort
pd.DataFrame(php_primary_count_pct)

primary_topic
name       primary_topic                                       
Arches     javascript-i                           0.149944     
           html-css                               0.115973     
           spring                                 0.102812     
           mysql                                  0.087177     
           java-iii                               0.078290     
           javascript-ii                          0.075591     
           jquery                                 0.073116     
                                                  0.070416     
           java-ii                                0.056243     
           java-i                                 0.047469     
           appendix                               0.044207     
           content                                0.031159     
           slides                                 0.024634     
           examples                               0.013611     
           toc                                    0.007424     
           mkdocs                                 0.006299     
           prework                                0.006187     
           search                                 0.005062     
           assets                                 0.001012     
           cohorts                                0.001012     
           5.0_Intro_Regression                   0.000225     
           7.0_Intro_Clustering                   0.000225     
           register.html                          0.000225     
           registration                           0.000225     
           students                               0.000225     
           uploads                                0.000225     
           0_Classification_Algorithms            0.000112     
           2.0_Intro_Stats                        0.000112     
           8.0_Intro_Module                       0.000112     
           About_NLP                              0.000112     
           Module_6_Classification                0.000112     
           index.html                             0.000112     
           introduction-to-matplotlib             0.000112     
           login                                  0.000112     
           pre-work                               0.000112     
Badlands   content                                0.612903     
                                                  0.182796     
           prework                                0.064516     
           javascript-ii                          0.053763     
           html-css                               0.043011     
           toc                                    0.021505     
           java-i                                 0.010753     
           javascript-i                           0.010753     
Denali     prework                                0.500000     
                                                  0.250000     
           mkdocs                                 0.250000     
Everglades                                        1.000000     
Franklin   javascript-ii                          0.180556     
           java-iii                               0.152778     
           javascript-i                           0.111111     
           java-ii                                0.069444     
           mysql                                  0.069444     
           spring                                 0.069444     
                                                  0.055556     
           java-i                                 0.055556     
           jquery                                 0.041667     
           11._DistributedML                      0.013889     
           12.01_SocialNetworkAnalysis            0.013889     
           2.00.02_Navigating_Excel               0.013889     
           2.00.05_Charts_PivotTables_Sparklines  0.013889     
           2.02.00_Inferential_Stats              0.013889     
           3.9-tem

In [12]:
# Full-Stack PHP: Count of Lessons Accessed by Cohort
php_sub_count = (full_stack_php.groupby('name').subtopic.value_counts())

In [13]:
# Full-Stack PHP: Percentage of Lessons Accessed by Cohort
php_sub_count_pct = (full_stack_php.groupby('name').subtopic.value_counts(normalize=True))

In [14]:
# Dataframe of Count of Lessons Accessed by Cohort
pd.DataFrame(php_sub_count)

subtopic
name       subtopic                                 
Arches     None                             1803    
                                            626     
           fundamentals                     574     
           introduction                     362     
           css-ii                           320     
           css-i                            274     
           arrays                           200     
           bom-and-dom                      195     
           relationships                    133     
           ajax                             120     
           laravel                          110     
           clauses                          108     
           functions                        107     
           search_index.json                101     
           mvc                              100     
           npm                              96      
           jdbc                             92      
           further-reading                  90      
           elements                         87      
           promises                         85      
           essential-methods                82      
           conditionals                     81      
           events                           80      
           servlets                         79      
           jsp-and-jstl                     78      
           loops                            78      
           map-filter-reduce                77      
           extra-features                   76      
           forms                            76      
           javascript-with-html             76      
           object-oriented-programming      75      
           setup                            73      
           modules                          72      
           ajax-api                         69      
           effects                          69      
           objects                          69      
           es6                              68      
           methods                          64      
           google-maps-api                  61      
           javascript                       60      
           css                              58      
           user-input                       58      
           syntax-types-and-variables       55      
           sessions-and-cookies             54      
           database-design                  53      
           introduction-to-java             53      
           strings                          52      
           capstone-workbook                50      
           aliases                          49      
           collections                      49      
           passwords                        49      
           personal-site                    47      
           php_iii                          47      
           cli                              46      
           deployment-and-dependencies      46      
           console-io                       45      
           group-by                         45      
           coffee-project                   44      
           control-statements-and-loops     41      
           interfaces-and-abstract-classes  41      
           selectors                        41      
           databases                        40      
           inheritance-and-polymorphism     39      
           basic-statements                 38      
           users                            37      
           tables                           36      
           finish-the-adlister              35      
           intellij                         35      
           annotations                      34      
           git                              34      
           sample-database                  34      
           html-css                         32      
           simple-simon                     32      
           extra-challenges                 31      
           excep

In [15]:
# Dataframe of Percentage of Lessons Accessed by Cohort
pd.DataFrame(php_sub_count_pct)

subtopic
name       subtopic                                 
Arches     None                             0.202812
                                            0.070416
           fundamentals                     0.064567
           introduction                     0.040720
           css-ii                           0.035996
           css-i                            0.030821
           arrays                           0.022497
           bom-and-dom                      0.021935
           relationships                    0.014961
           ajax                             0.013498
           laravel                          0.012373
           clauses                          0.012148
           functions                        0.012036
           search_index.json                0.011361
           mvc                              0.011249
           npm                              0.010799
           jdbc                             0.010349
           further-reading                  0.010124
           elements                         0.009786
           promises                         0.009561
           essential-methods                0.009224
           conditionals                     0.009111
           events                           0.008999
           servlets                         0.008886
           jsp-and-jstl                     0.008774
           loops                            0.008774
           map-filter-reduce                0.008661
           extra-features                   0.008549
           forms                            0.008549
           javascript-with-html             0.008549
           object-oriented-programming      0.008436
           setup                            0.008211
           modules                          0.008099
           ajax-api                         0.007762
           effects                          0.007762
           objects                          0.007762
           es6                              0.007649
           methods                          0.007199
           google-maps-api                  0.006862
           javascript                       0.006749
           css                              0.006524
           user-input                       0.006524
           syntax-types-and-variables       0.006187
           sessions-and-cookies             0.006074
           database-design                  0.005962
           introduction-to-java             0.005962
           strings                          0.005849
           capstone-workbook                0.005624
           aliases                          0.005512
           collections                      0.005512
           passwords                        0.005512
           personal-site                    0.005287
           php_iii                          0.005287
           cli                              0.005174
           deployment-and-dependencies      0.005174
           console-io                       0.005062
           group-by                         0.005062
           coffee-project                   0.004949
           control-statements-and-loops     0.004612
           interfaces-and-abstract-classes  0.004612
           selectors                        0.004612
           databases                        0.004499
           inheritance-and-polymorphism     0.004387
           basic-statements                 0.004274
           users                            0.004162
           tables                           0.004049
           finish-the-adlister              0.003937
           intellij                         0.003937
           annotations                      0.003825
           git                              0.003825
           sample-database                  0.003825
           html-css                         0.003600
           simple-simon                     0.003600
           extra-challenges                 0.003487
           excep

### Full-Stack Java Program

In [16]:
# Full-Stack Java: Count of Modules Accessed by Cohort
java_primary_count = (full_stack_java.groupby('name').primary_topic.value_counts())

In [17]:
# Full-Stack Java: Percentage of Modules Accessed by Cohort
java_primary_count_pct = (full_stack_java.groupby('name').primary_topic.value_counts(normalize=True))

In [18]:
# Dataframe of Count of Modules Accessed by Cohort
pd.DataFrame(java_primary_count)

primary_topic
name       primary_topic                                       
Andromeda  javascript-i                           4077         
           mysql                                  3029         
           html-css                               2865         
           spring                                 2850         
           java-iii                               2117         
           jquery                                 2101         
           java-ii                                1884         
           javascript-ii                          1340         
           java-i                                 1330         
                                                  1174         
           appendix                               796          
           toc                                    638          
           examples                               530          
           search                                 318          
           content                                143          
           web-design                             66           
           prework                                25           
           slides                                 21           
           assets                                 14           
           1-fundamentals                         6            
           arrays                                 3            
           deployment-and-dependencies            3            
           forms                                  3            
           login                                  3            
           students                               3            
           7-classification                       2            
           loops                                  2            
           objects                                2            
           setup                                  2            
           3-sql                                  1            
           bonus-exercises                        1            
           capstone                               1            
           documentation                          1            
           events                                 1            
           html                                   1            
           htmle-css                              1            
           jsp-and-jstl                           1            
           quiz                                   1            
           sessions-and-cookies                   1            
           testing-user-functions                 1            
           tools                                  1            
Apex       javascript-i                           4436         
           mysql                                  3835         
           html-css                               3639         
           java-ii                                3225         
           spring                                 2684         
           java-iii                               2645         
           jquery                                 2570         
           java-i                                 2231         
           appendix                               2042         
           javascript-ii                          1790         
           search                                 1497         
                                                  1346         
           toc                                    810          
           examples                               451          
           web-design                             146          
           prework                                62           
           capstone                               60           
           slides                                 52           
           elements                               23           
           finish-the-adlister                    8            
           css-i  

In [19]:
# Dataframe of Percentage of Modules Accessed by Cohort
pd.DataFrame(java_primary_count_pct)

primary_topic
name       primary_topic                                       
Andromeda  javascript-i                           0.160771     
           mysql                                  0.119445     
           html-css                               0.112978     
           spring                                 0.112386     
           java-iii                               0.083481     
           jquery                                 0.082850     
           java-ii                                0.074293     
           javascript-ii                          0.052841     
           java-i                                 0.052447     
                                                  0.046295     
           appendix                               0.031389     
           toc                                    0.025159     
           examples                               0.020900     
           search                                 0.012540     
           content                                0.005639     
           web-design                             0.002603     
           prework                                0.000986     
           slides                                 0.000828     
           assets                                 0.000552     
           1-fundamentals                         0.000237     
           arrays                                 0.000118     
           deployment-and-dependencies            0.000118     
           forms                                  0.000118     
           login                                  0.000118     
           students                               0.000118     
           7-classification                       0.000079     
           loops                                  0.000079     
           objects                                0.000079     
           setup                                  0.000079     
           3-sql                                  0.000039     
           bonus-exercises                        0.000039     
           capstone                               0.000039     
           documentation                          0.000039     
           events                                 0.000039     
           html                                   0.000039     
           htmle-css                              0.000039     
           jsp-and-jstl                           0.000039     
           quiz                                   0.000039     
           sessions-and-cookies                   0.000039     
           testing-user-functions                 0.000039     
           tools                                  0.000039     
Apex       javascript-i                           0.132150     
           mysql                                  0.114246     
           html-css                               0.108407     
           java-ii                                0.096074     
           spring                                 0.079957     
           java-iii                               0.078795     
           jquery                                 0.076561     
           java-i                                 0.066462     
           appendix                               0.060832     
           javascript-ii                          0.053325     
           search                                 0.044596     
                                                  0.040098     
           toc                                    0.024130     
           examples                               0.013435     
           web-design                             0.004349     
           prework                                0.001847     
           capstone                               0.001787     
           slides                                 0.001549     
           elements                               0.000685     
           finish-the-adlister                    0.000238     
           css-i  

In [20]:
# Full-Stack Java: Count of Lessons Accessed by Cohort
# Full-Stack Java: Percentage of Lessons Accessed by Cohort
java_sub_count = (full_stack_java.groupby('name').subtopic.value_counts())
java_sub_count_pct = (full_stack_java.groupby('name').subtopic.value_counts(normalize=True))

In [21]:
# Dataframe of Count of Lessons Accessed by Cohort
pd.DataFrame(java_sub_count)

subtopic
name       subtopic                                                 
Andromeda  None                                             4381    
           fundamentals                                     1757    
           introduction                                     1387    
                                                            1176    
           css-i                                            982     
           css-ii                                           916     
           arrays                                           570     
           bom-and-dom                                      449     
           relationships                                    440     
           events                                           422     
           ajax                                             413     
           objects                                          390     
           extra-features                                   363     
           functions                                        339     
           clauses                                          325     
           essential-methods                                325     
           search_index.json                                318     
           tables                                           318     
           databases                                        292     
           css                                              278     
           jsp-and-jstl                                     274     
           servlets                                         270     
           javascript-with-html                             257     
           loops                                            254     
           elements                                         253     
           object-oriented-programming                      242     
           users                                            242     
           further-reading                                  239     
           interfaces-and-abstract-classes                  238     
           inheritance-and-polymorphism                     232     
           mvc                                              232     
           forms                                            224     
           setup                                            221     
           basic-statements                                 209     
           ajax-api                                         208     
           conditionals                                     206     
           mapbox-api                                       204     
           jdbc                                             200     
           syntax-types-and-variables                       197     
           promises                                         194     
           selectors                                        191     
           collections                                      185     
           methods                                          181     
           javascript                                       178     
           deployment-and-dependencies                      176     
           console-io                                       174     
           introduction-to-java                             170     
           modules                                          165     
           es6                                              161     
           user-input                                       161     
           control-statements-and-loops                     159     
           map-filter-reduce                                156     
           npm                                              155     
           sessions-and-cookies                             152     
           coffee-project                                   151     
           effects                                          147     
           file-io                                        

In [22]:
# Dataframe of Percentage of Lessons Accessed by Cohort
pd.DataFrame(java_sub_count_pct)

subtopic
name       subtopic                                                 
Andromeda  None                                             0.172759
           fundamentals                                     0.069285
           introduction                                     0.054695
                                                            0.046374
           css-i                                            0.038724
           css-ii                                           0.036121
           arrays                                           0.022477
           bom-and-dom                                      0.017706
           relationships                                    0.017351
           events                                           0.016641
           ajax                                             0.016286
           objects                                          0.015379
           extra-features                                   0.014314
           functions                                        0.013368
           clauses                                          0.012816
           essential-methods                                0.012816
           search_index.json                                0.012540
           tables                                           0.012540
           databases                                        0.011515
           css                                              0.010963
           jsp-and-jstl                                     0.010805
           servlets                                         0.010647
           javascript-with-html                             0.010134
           loops                                            0.010016
           elements                                         0.009977
           object-oriented-programming                      0.009543
           users                                            0.009543
           further-reading                                  0.009425
           interfaces-and-abstract-classes                  0.009385
           inheritance-and-polymorphism                     0.009149
           mvc                                              0.009149
           forms                                            0.008833
           setup                                            0.008715
           basic-statements                                 0.008242
           ajax-api                                         0.008202
           conditionals                                     0.008123
           mapbox-api                                       0.008044
           jdbc                                             0.007887
           syntax-types-and-variables                       0.007768
           promises                                         0.007650
           selectors                                        0.007532
           collections                                      0.007295
           methods                                          0.007138
           javascript                                       0.007019
           deployment-and-dependencies                      0.006940
           console-io                                       0.006861
           introduction-to-java                             0.006704
           modules                                          0.006507
           es6                                              0.006349
           user-input                                       0.006349
           control-statements-and-loops                     0.006270
           map-filter-reduce                                0.006152
           npm                                              0.006112
           sessions-and-cookies                             0.005994
           coffee-project                                   0.005954
           effects                                          0.005797
           file-io                                        

### Data Science Program

In [23]:
# Data Science: Count of Lessons Accessed by Cohort
ds_sub_count = (data_science.groupby('name').subtopic.value_counts())

In [24]:
# Data Science: Percentage of Lessons Accessed by Cohort
ds_sub_count_pct = (data_science.groupby('name').primary_topic.value_counts(normalize=True))

In [25]:
# Dataframe of Count of Lessons Accessed by Cohort
pd.DataFrame(ds_sub_count)

subtopic
name     subtopic                                                                                                                                                                                                                                                                                                             
Bayes                                                                                                                                                                                                                                                                                                                 1967    
         1-overview                                                                                                                                                                                                                                                                                                   1826    
         project                                                                                                                                                                                                                                                                                                      1024    
         modern-data-scientist.jpg                                                                                                                                                                                                                                                                                    674     
         AI-ML-DL-timeline.jpg                                                                                                                                                                                                                                                                                        672     
         4-explore                                                                                                                                                                                                                                                                                                    652     
         1.1-intro-to-data-science                                                                                                                                                                                                                                                                                    640     
         search_index.json                                                                                                                                                                                                                                                                                            589     
         cli                                                                                                                                                                                                                                                                                                          553     
         3-prep                                                                                                                                                                                                                                                                                                       431     
         AnomalyDetectionCartoon.jpeg                                                                                                                                                                                                                                                                                 401     
         5.0-evaluate                                                                                                                                              

In [26]:
# Dataframe of Percentage of Lessons Accessed by Cohort
pd.DataFrame(ds_sub_count_pct)

primary_topic
name     primary_topic                                                
Bayes    1-fundamentals                                  0.117718     
         6-regression                                    0.096955     
         4-python                                        0.090135     
         3-sql                                           0.087158     
                                                         0.074120     
         7-classification                                0.073291     
         5-stats                                         0.072424     
         8-clustering                                    0.050192     
         appendix                                        0.046914     
         10-anomaly-detection                            0.045218     
         9-timeseries                                    0.042467     
         11-nlp                                          0.042128     
         2-storytelling                                  0.030711     
         12-distributed-ml                               0.023363     
         13-advanced-topics                              0.022760     
         search                                          0.022232     
         classification                                  0.009797     
         sql                                             0.007762     
         fundamentals                                    0.007725     
         python                                          0.007197     
         regression                                      0.006067     
         stats                                           0.004333     
         clustering                                      0.003693     
         timeseries                                      0.002939     
         anomaly-detection                               0.001922     
         nlp                                             0.001432     
         storytelling                                    0.001243     
         html-css                                        0.000980     
         distributed-ml                                  0.000829     
         advanced-topics                                 0.000490     
         3-vocabulary.md                                 0.000452     
         javascript-i                                    0.000452     
         3-discrete-probabilistic-methods                0.000339     
         toc                                             0.000339     
         java-i                                          0.000301     
         spring                                          0.000264     
         login                                           0.000226     
         Hospital-Distance-Clusters.jpg                  0.000188     
         Pipeline_Demo                                   0.000151     
         bayes-python-assessment-4586.html               0.000151     
         cli                                             0.000151     
         find_me_curie!(12.3.19)                         0.000151     
         java-ii                                         0.000151     
         java-iii                                        0.000151     
         javascript-ii                                   0.000151     
         jquery                                          0.000151     
         13.1_Understand                                 0.000113     
         capstones                                       0.000113     
         mysql                                           0.000113     
         scale_features_or_not.svg                       0.000113     
         1-overview                                      0.000075     
         4-detecting-timeseries-anomalies                0.000075     
         Selecting_a_hypothesis_test.svg                 0.000075     
         classical_programming_vs_machine_learning.jpeg  0.000075     
         examples                                        0.000075     
         l

### Question 1: Answer
From our exploration, we've determined that the lessons that attract the most traffic consistently across cohorts are the fundamentals and concept introduction pages. Regardless of program, it seems that these initial pages, and associated fundamental concepts are most visited across cohorts.

-----------------------------------------------------------------------------------------------------------
## 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?


Analysis performed above answering the first question can also be used to answer this second question. 

**Takeaways: full stack php**
1. laravel is referred to significantly more by 'Kings' cohort than other cohorts; most of whom gloss over it.

**Takeaways: full stack java**
1. No outstanding differences within full stack java cohorts (all are rather uniform)

**Takeaways: data science**
1. Cohorts Darden and Easley referred to 'scale_features_or_not' significantly more than other cohorts

### Question 2: Answer
From our exploration, we've determined:
- Full-Stack Java Cohorts: No outstanding differences between cohorts (all are uniform)
- Full-Stack PHP Cohorts: Kings cohort refers to 'laravel' significantly more than other cohorts
- Data Science Cohorts: Darden and Easley cohorts refer to 'scale_features_or_not' significantly more than other cohorts

-----------------------------------------------------------------------------------------------------------
## 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [27]:
# Creating df that only pulls data of students who are actively in their class period
active_students = df[(df.date <= df.end_date)|(df.date >= df.start_date)]

In [28]:
# describe value counts of user_id, shows hits
active_students.user_id.value_counts().describe()

count    911.000000  
mean     930.108672  
std      1236.014277 
min      1.000000    
25%      193.000000  
50%      697.000000  
75%      1298.500000 
max      17913.000000
Name: user_id, dtype: float64

In [29]:
# Because the user_id is used on multiple rows, set it as index so it can be explored
active_students = active_students.set_index('user_id')

In [30]:
# Simplify dataframe, drop unused columns
active_students = active_students.drop(columns=['start_date','end_date','created_at','date','time'])

In [31]:
#  Shows simplified df
active_students.head()

,path,cohort_id,ip,name,program_id,primary_topic,subtopic,tertiary
user_id,,,,,,,,
1,/,8.0,97.105.19.61,Hampton,1.0,,,None
1,java-ii,8.0,97.105.19.61,Hampton,1.0,java-ii,None,None
1,java-ii/object-oriented-programming,8.0,97.105.19.61,Hampton,1.0,java-ii,object-oriented-programming,None
1,slides/object_oriented_programming,8.0,97.105.19.61,Hampton,1.0,slides,object_oriented_programming,None
2,javascript-i/conditionals,22.0,97.105.19.61,Teddy,2.0,javascript-i,conditionals,None


In [45]:
# Utilizing simplified dataframe, we setup where we draw the line for limited use of active students, around 150 hits, below the lower IQR of 193
low_use_active_students = active_students[active_students.index.value_counts() <=150]
low_use_active_students.head()

,path,cohort_id,ip,name,program_id,primary_topic,subtopic,tertiary
user_id,,,,,,,,
13,javascript-i/functions,22.0,97.105.19.61,Teddy,2.0,javascript-i,functions,None
21,index.html,16.0,71.137.146.71,Niagara,2.0,index.html,None,None
21,java-i,16.0,71.137.146.71,Niagara,2.0,java-i,None,None
24,appendix,21.0,97.105.19.61,Sequoia,2.0,appendix,None,None
24,appendix/capstone-workbook,21.0,97.105.19.61,Sequoia,2.0,appendix,capstone-workbook,None


In [44]:
# Visualizing simplified dataframe showing low use active students by cohort 
(pd.DataFrame(active_students[active_students.index.value_counts() <=150].groupby('user_id').name.value_counts())).sort_index()

,,name
user_id,name,
13,Teddy,94
21,Niagara,107
24,Sequoia,26
45,Sequoia,65
46,Teddy,54
49,Sequoia,25
55,Sequoia,95
65,Sequoia,107
66,Kings,1


### Question 3: Answer
From our exploration, we've determined that 

Can we see how often these same students utilized the curriculum after they graduated?

-----------------------------------------------------------------------------------------------------------
## 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

### Question 5: Answer
From our exploration, we've determined that 

-----------------------------------------------------------------------------------------------------------
## 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

### Question 6: Answer
From our exploration, we've determined that 

-----------------------------------------------------------------------------------------------------------
## 7. Which lessons are least accessed?

In [ ]:
# Full-Stack PHP: Percentage of Lessons Accessed by Cohort
php_sub_count_pct = (full_stack_php.groupby('name').subtopic.value_counts(normalize=True, ascending=True))
pd.DataFrame(php_sub_count_pct)

subtopic
name       subtopic                                 
Arches     getting-started                  0.000112
           mapbox-api                       0.000112
           promisesdfghjkjhgfs              0.000112
           units                            0.000112
           5762c2946250b.jpg                0.000225
           exceptions_and_error_handling    0.000225
           file_io                          0.000225
           versioning                       0.000225
           vocab                            0.000225
           javascript_ii                    0.000337
           alumni-tech-survey-2018          0.000450
           appendix                         0.000450
           slides                           0.000450
           deployment_and_dependencies      0.000562
           interfaces_and_abstract_classes  0.000562
           java                             0.000562
           php_i                            0.000675
           php_ii                           0.000900
           syntax_types_and_variables       0.000900
           24                               0.001012
           bootstrap-grid                   0.001012
           documentation                    0.001012
           js                               0.001012
           sessions_and_cookies             0.001012
           whack-a-mole                     0.001012
           console_io                       0.001125
           introduction_to_java             0.001125
           user_input                       0.001125
           angular                          0.001237
           control_statements_and_loops     0.001237
           extra-exercises                  0.001237
           inheritance_and_polymorphism     0.001237
           jsp_and_jstl                     0.001237
           code-standards                   0.001462
           object_oriented_programming      0.001687
           postwork                         0.002250
           php_iv                           0.002925
           file-io                          0.003150
           mysql                            0.003150
           exceptions-and-error-handling    0.003262
           extra-challenges                 0.003487
           html-css                         0.003600
           simple-simon                     0.003600
           annotations                      0.003825
           git                              0.003825
           sample-database                  0.003825
           finish-the-adlister              0.003937
           intellij                         0.003937
           tables                           0.004049
           users                            0.004162
           basic-statements                 0.004274
           inheritance-and-polymorphism     0.004387
           databases                        0.004499
           control-statements-and-loops     0.004612
           interfaces-and-abstract-classes  0.004612
           selectors                        0.004612
           coffee-project                   0.004949
           console-io                       0.005062
           group-by                         0.005062
           cli                              0.005174
           deployment-and-dependencies      0.005174
           personal-site                    0.005287
           php_iii                          0.005287
           aliases                          0.005512
           collections                      0.005512
           passwords                        0.005512
           capstone-workbook                0.005624
           strings                          0.005849
           database-design                  0.005962
           introduction-to-java             0.005962
           sessions-and-cookies             0.006074
           syntax-types-and-variables       0.006187
           css                              0.006524
           user-input                       0.006524
           javas

In [ ]:
# Full-Stack Java: Percentage of Lessons Accessed by Cohort
java_sub_count_pct = (full_stack_java.groupby('name').subtopic.value_counts(normalize=True))

In [ ]:
# Data Science: Percentage of Lessons Accessed by Cohort
ds_sub_count_pct = (data_science.groupby('name').primary_topic.value_counts(normalize=True))

### Question 7: Answer
From our exploration, we've determined that 